In [18]:
import stageemi
import stageemi.dev.visu as dev
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl

import stageemi.dev.decorator_map as dm
import ipywidgets as widg 
import ipyleaflet as ipyl
import datetime as dt
import os 
from ipywidgets import Text, HTML
from ipyleaflet import WidgetControl
import pandas as pd
from datetime import datetime
import glob 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics.cluster import contingency_matrix
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [19]:
dpt_nb=["34","41","29","38"]
zone_dpt=['FRJ13','FRB05','FRH02','FRK24'] #['FRB05' Loir-et-Cher,"FRH02" Finistere,'FRJ13' Herault,'FRK24' Isere]
zone_dpt

['FRJ13', 'FRB05', 'FRH02', 'FRK24']

In [20]:
dpt_choice=dpt_nb[0]
zone_sympo=[zone.split("ZONE_SYMPO/")[1].split(".nc")[0] for i,zone in enumerate(sorted(glob.glob("../../../StageEMI_masks/Masques_netcdf/ZONE_SYMPO/"+dpt_choice+"*")))]
zone_sympo

['3401',
 '3402',
 '3403',
 '3404',
 '3405',
 '3406',
 '3407',
 '3408',
 '3481',
 '3482',
 '3483']

In [21]:
# choose between zone_dpt and zone_sympo
zone_to_process=zone_dpt

In [22]:
path="./scores/"
dpt_choice=dpt_nb[0]
zone_sympo=np.unique([zone.split(path[2:])[1].split("_scores")[0] for i,zone in enumerate(sorted(glob.glob(path+"*"+dpt_choice+"*_*")))])
zone_sympo

array(['3401', '3402', '3403', '3404', '3405', '3406', '3407', '3408',
       '3481', '3482', '3483'], dtype='<U4')

In [23]:
path_figures="./scores_figures/"

In [24]:
file_CodesWWMF = '../utils/CodesWWMF.csv'
df = pd.read_csv(file_CodesWWMF,usecols = (0,1,2,3,6,7),sep=',')
code_WWMF    = df['Code WWMF'].to_numpy()

In [25]:
for izone, zone in enumerate(zone_to_process):
                   
    print('Currently processing zone: '+zone)

    files=sorted(glob.glob(path+"*"+zone+"*"))        

    scores=pd.read_csv(files[0],sep=',',index_col=0)
#     print('Currently reading file: '+files[0])        

    for ifile,fname in enumerate(files[1:]):    
        df_scores=pd.read_csv(fname,sep=',',index_col=0)
#         print('Currently reading file: '+fname)

        scores=scores.append(df_scores)   

        wwmf_codes=list(np.unique(scores["WWMF"]))
        scores["WWMF to WME"]='default'
        scores["WWMF to W1"]='default'

        for iwwmf,wwmf in enumerate(wwmf_codes):    
            scores["WWMF to WME"].loc[scores["WWMF"]==wwmf]=df[df["Code WWMF"]==wwmf]["Code WME"].iloc[0]    
            scores["WWMF to W1"].loc[scores["WWMF"]==wwmf]=df[df["Code WWMF"]==wwmf]["Code W1"].iloc[0]

    # process WME 

    true_wme=np.unique(scores["WWMF to WME"])
    pred_wme=np.unique(scores["WME"])
    l_wme=len(list(set().union(true_wme,pred_wme)))
    labels_wme=list(set().union(true_wme,pred_wme))
    matrix_wme=confusion_matrix(scores["WWMF to WME"],scores["WME"])
    # matrix_wme
    div_wme=np.ones((l_wme,1))*matrix_wme.sum(axis=0)
    matrix_wme_norm=np.nan_to_num(matrix_wme/div_wme)
    # matrix_wme_norm
    result_wme= ConfusionMatrixDisplay(matrix_wme,display_labels=labels_wme)
    result_wme_norm= ConfusionMatrixDisplay(matrix_wme_norm,display_labels=labels_wme)
    print('Number of elements (WME confusion matrix): '+str(matrix_wme.sum())) 

    fig = plt.figure(figsize=(20,8))
    ax = fig.add_subplot(1,2,1)
    result_wme.plot(ax=ax,values_format=".0f",cmap=plt.cm.Blues)
    plt.xlabel("Predicted label",fontsize=20)
    plt.ylabel("True label",fontsize=20)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Confusion matrix WME/COMPAS: "+zone+"\n",fontsize=20)
    ax = fig.add_subplot(1,2,2)
    result_wme_norm.plot(ax=ax,values_format=".02f",cmap=plt.cm.Blues)
    plt.xlabel("Predicted label",fontsize=20)
    plt.ylabel("True label",fontsize=20)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Confusion matrix WME/COMPAS (norm): "+zone+"\n",fontsize=20)
    plt.tight_layout()
    plt.savefig(path_figures+'confusion_matrix_WME_'+zone+'.png',bbox_inches = 'tight')
    plt.close()

    # display bar plot for errors
    correct_wme=100*np.nan_to_num(np.diag(matrix_wme)/matrix_wme.sum(axis=0))
    error_wme=100*np.nan_to_num((matrix_wme.sum(axis=0)-np.diag(matrix_wme))/matrix_wme.sum(axis=0))

    w_wme=matrix_wme.sum(axis=0)/matrix_wme.sum()
    success_wme=sum(np.multiply(correct_wme,w_wme))

    fig = plt.figure(figsize=(10,10))    
    ax = fig.add_subplot(1,1,1)
    ind_wme = np.arange(len(error_wme))    # the x locations for the groups
    width = 0.35       # the width of the bars: can also be len(x) sequence
    p1 = plt.bar(ind_wme, correct_wme, width)
    p2 = plt.bar(ind_wme, error_wme, width,bottom=correct_wme)
    plt.axhline(y=90, color='r', linestyle='--')
    plt.axhline(y=success_wme, color='g', linestyle='--')
    plt.xlabel('Predicted label',fontsize=20)
    plt.ylabel('Success rate (%)',fontsize=20)
    plt.title('Success rate='+'{0:.2f}'.format(success_wme)+' - WME/COMPAS: '+' '+zone+"\n",fontsize=20)
    plt.xticks(ind_wme, labels_wme,fontsize=15)
    plt.yticks(np.arange(0, 110, 10),fontsize=15)
    plt.ylim((0,110))
    plt.legend((p1[0], p2[0]), ('Correct', 'Incorrect'),fontsize=15,loc='upper right')
    plt.savefig(path_figures+'success_rate_WME_'+zone+'.png',bbox_inches = 'tight') 
    plt.close()


    # process W1

    true_w1=list(np.unique(scores["WWMF to W1"]))
    pred_w1=list(np.unique(scores["W1"]))
    l_w1=len(list(set().union(true_w1,pred_w1)))
    labels_w1=list(set().union(true_w1,pred_w1))
    matrix_w1=confusion_matrix(scores["WWMF to W1"],scores["W1"])
    # matrix_w1
    div_w1=np.ones((l_w1,1))*matrix_w1.sum(axis=0)
    matrix_w1_norm=np.nan_to_num(matrix_w1/div_w1)
    result_w1= ConfusionMatrixDisplay(matrix_w1,display_labels=labels_w1)
    result_w1_norm= ConfusionMatrixDisplay(matrix_w1_norm,display_labels=labels_w1)
    print('Number of elements (W1 confusion matrix): '+str(matrix_w1.sum())) 

    fig = plt.figure(figsize=(20,8))
    ax = fig.add_subplot(1,2,1)
    result_w1.plot(ax=ax,values_format=".0f",cmap=plt.cm.Blues)
    plt.xlabel("Predicted label",fontsize=20)
    plt.ylabel("True label",fontsize=20)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Confusion matrix W1/AGAT: "+zone+"\n",fontsize=20)
    ax = fig.add_subplot(1,2,2)
    result_w1_norm.plot(ax=ax,values_format=".02f",cmap=plt.cm.Blues)
    plt.xlabel("Predicted label",fontsize=20)
    plt.ylabel("True label",fontsize=20)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Confusion matrix W1/AGAT (norm): "+zone+"\n",fontsize=20)
    plt.tight_layout()
    plt.savefig(path_figures+'confusion_matrix_W1_'+zone+'.png',bbox_inches = 'tight')            
    plt.close()

    # display bar plot for errors
    correct_w1=100*np.nan_to_num(np.diag(matrix_w1)/matrix_w1.sum(axis=0))
    error_w1=100*np.nan_to_num((matrix_w1.sum(axis=0)-np.diag(matrix_w1))/matrix_w1.sum(axis=0))

    w_w1=matrix_w1.sum(axis=0)/matrix_w1.sum()
    success_w1=sum(np.multiply(correct_w1,w_w1))

    fig = plt.figure(figsize=(10,10))    
    ax = fig.add_subplot(1,1,1)
    ind_w1 = np.arange(len(error_w1))    # the x locations for the groups
    width = 0.35       # the width of the bars: can also be len(x) sequence
    p1 = plt.bar(ind_w1, correct_w1, width)
    p2 = plt.bar(ind_w1, error_w1, width,bottom=correct_w1)
    plt.axhline(y=90, color='r', linestyle='--')
    plt.axhline(y=success_w1, color='g', linestyle='--')
    plt.xlabel('Predicted label',fontsize=20)
    plt.ylabel('Success rate (%)',fontsize=20)
    plt.title('Success rate='+'{0:.2f}'.format(success_w1)+' - W1/AGAT: '+' '+zone+"\n",fontsize=20)
    plt.xticks(ind_w1, labels_w1,fontsize=15)
    plt.yticks(np.arange(0, 110, 10),fontsize=15)
    plt.ylim((0,110))
    plt.legend((p1[0], p2[0]), ('Correct', 'Incorrect'),fontsize=15,loc='upper right')
    plt.savefig(path_figures+'success_rate_W1_'+zone+'.png',bbox_inches = 'tight') 
    plt.close()
        
#     del scores

Currently processing zone: FRJ13


/opt/anaconda3/envs/preproc2/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in true_divide


Number of elements (WME confusion matrix): 4224


/opt/anaconda3/envs/preproc2/lib/python3.7/site-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in true_divide
/opt/anaconda3/envs/preproc2/lib/python3.7/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in true_divide
/opt/anaconda3/envs/preproc2/lib/python3.7/site-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in true_divide


Number of elements (W1 confusion matrix): 4224


/opt/anaconda3/envs/preproc2/lib/python3.7/site-packages/ipykernel_launcher.py:118: RuntimeWarning: invalid value encountered in true_divide
/opt/anaconda3/envs/preproc2/lib/python3.7/site-packages/ipykernel_launcher.py:119: RuntimeWarning: invalid value encountered in true_divide


Currently processing zone: FRB05
Number of elements (WME confusion matrix): 4224
Number of elements (W1 confusion matrix): 4224
Currently processing zone: FRH02
Number of elements (WME confusion matrix): 4224
Number of elements (W1 confusion matrix): 4224
Currently processing zone: FRK24
Number of elements (WME confusion matrix): 4224
Number of elements (W1 confusion matrix): 4224


In [6]:
np.arange(0, 110, 10)

array([  0,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100])

In [7]:
x = np.arange(9).reshape((3,3))
x

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [8]:
d=np.diag(x)

In [9]:
y=np.ones((3,))*4
y

array([4., 4., 4.])

In [10]:
d

array([0, 4, 8])

In [11]:
d-y

array([-4.,  0.,  4.])

In [12]:
# scores.iloc[0:20]
# print(len(scores))

In [13]:
# fig = plt.figure(figsize=(8, 6))
# ax1 = fig.add_subplot(111, projection='3d')

# colors = ['red','orange','green']

# for sc in [0,1,2]:
#     icol=[i for i, e in enumerate(scores['WWMF vs (WME and W1)']) if e == sc]

#     xs = scores.iloc[icol]['WWMF']
#     ys = scores.iloc[icol]['WME']
#     zs = scores.iloc[icol]['W1']

#     ax1.scatter(xs, ys, zs, alpha=0.4, c=colors[sc], s=30)
    
# ax1.set_xlabel('Majoritary WWMF')
# ax1.set_ylabel('Compas WME')
# ax1.set_zlabel('Agat W1')